In [1]:
#!/usr/bin/python
import os 
import sagemaker
import subprocess


# Define instance configurations 
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

repo_name = 'ocr' # ECR repository
image_tag = 'prj_scsk' # ECR image tag
base_job_name = 'scsk-mixnet-ocr' # SageMaker training prefix
# dockerfile = os.path.abspath('./new_dockerfile')

%env dockerfile Dockerfile
%env account {account}
%env region {region}
%env repo_name {repo_name}
%env image_tag {image_tag}

# print("Account: {0}".format(account))
# print("Region: {0}".format(region))
# print("Repo name: {0}".format(repo_name))
# print("Image tag: {0}".format(image_tag))
# print("Base job name: {0}".format(base_job_name))
# print("Docker file: {0}".format(dockerfile))

env: dockerfile=Dockerfile
env: account=533155507761
env: region=us-west-2
env: repo_name=ocr
env: image_tag=prj_scsk


In [2]:
%%sh
aws ecr describe-repositories --repository-names $repo_name > /dev/null 2>&1
if [ $? -ne 0 ]
then
   aws ecr create-repository --repository-name $repo_name > /dev/null
fi
$(aws ecr get-login --region $region --no-include-email)

Login Succeeded


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [3]:
# # Build docker and push to ionstance
# subprocess.run("docker build -t {0} -f {1} . ".format(image_tag, dockerfile), shell=True)
# subprocess.run("docker tag {0} {1}.dkr.ecr.{2}.amazonaws.com/{3}:latest".format(image_tag, account, region, repo_name), shell=True)
# subprocess.run("docker push {0}.dkr.ecr.{1}.amazonaws.com/{2}:latest".format(account, region, repo_name), shell=True)

!docker build -t $image_tag -f $dockerfile .
!docker tag $image_tag $account.dkr.ecr.$region.amazonaws.com/$repo_name:latest
!docker images
!docker push $account.dkr.ecr.$region.amazonaws.com/$repo_name:latest

Sending build context to Docker daemon  820.2kB
Step 1/19 : FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04
 ---> a686b859d874
Step 2/19 : ARG py_version=3
 ---> Using cache
 ---> 17fdf7451d15
Step 3/19 : RUN test $py_version || exit 1
 ---> Using cache
 ---> 5e1bd5bcd004
Step 4/19 : RUN apt-get update && apt-get install -y --no-install-recommends software-properties-common &&     add-apt-repository ppa:deadsnakes/ppa -y &&     apt-get update && apt-get install -y --no-install-recommends         build-essential         cmake         curl         jq         libsm6         libxext6         libxrender-dev         nginx &&     if [ $py_version -eq 3 ];        then apt-get install -y --no-install-recommends python3.6-dev            && ln -s -f /usr/bin/python3.6 /usr/bin/python;        else apt-get install -y --no-install-recommends python-dev; fi &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> c6c33a8f5a68
Step 5/19 : RUN cd /tmp &&     curl -O https://bootstrap.pypa.io/get-pip.py

In [4]:

# Define data path in S3 
s3_directory = 's3://scsk-data/ocr_data/data'
train_input_channel = sagemaker.session.s3_input(s3_directory, distribution='FullyReplicated',  s3_data_type='S3Prefix')

# Define image name, output path to save model 
output_path = 's3://scsk-data/ocr_data/output/'
image_name  = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, repo_name)

## Define instance to train 
train_instance_type = 'ml.p3.2xlarge'
# train_instance_type = 'ml.p3.8xlarge'

# Define space of disk to storage input data
storage_space = 200 # Gb

# Maximum seconds for this training job’s life (days * hours * seconds)
train_max_run = 3 * 24  * 3600


In [ ]:

# Set sagemaker estimator and process to train
estimator = sagemaker.estimator.Estimator(
                       image_name=image_name,
                       base_job_name=base_job_name,
                       role=role,
                       input_mode='File',
                       train_instance_count=1,
                       train_volume_size=storage_space,
                       train_instance_type=train_instance_type,
                       output_path=output_path,
                       train_max_run=train_max_run,
                       sagemaker_session=sess)

estimator.fit({'train': train_input_channel})

2019-08-11 13:24:10 Starting - Starting the training job...
2019-08-11 13:24:11 Starting - Launching requested ML instances.........
2019-08-11 13:25:42 Starting - Preparing the instances for training...
2019-08-11 13:26:29 Downloading - Downloading input data...
2019-08-11 13:27:10 Training - Downloading the training image........
Installing requirements...

2019-08-11 13:28:14 Training - Training image download completed. Training in progress.Collecting keras==2.2.4 (from -r requirements.txt (line 2))
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7541 sha256=81184e38a0a8ffcc94fcf30608ca11a197d3eb05ce5ff698a61dfb3288c9b932
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
  Created wheel for termcolor: filename=termcolor-1.1.0-cp36-none-any.whl size=4833 sha256=6c9c8f79752a41e89ffe0cc7f7886e08816d516394e846bbd2d2fe1ef3421b9a
  Stored in directory: /root/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b